# Transfer Learning

### Import Libraries

In [44]:
import pathlib
import os
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import keras_tuner as kt
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.model_selection import StratifiedKFold, train_test_split


2024-10-31 17:42:26.817391: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-31 17:42:26.900256: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-31 17:42:26.930681: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-31 17:42:26.941767: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-31 17:42:26.995435: I tensorflow/core/platform/cpu_feature_guar

## Train with MonkeyPox Dataset

### Hyperparameters

In [ ]:
data_root = pathlib.Path("../data/MSLD/Augmented_Images")    # points to the folder containing the images that will be used for training

# hyperparameters
img_height = 224        # input image height
img_width = 224         # input image width
batch_size = 16         # size of the batch that will be fed to model

# folds = the amount of folds that will be created for cross-validation
# fine_tune_epochs = number of epochs after which we start fine-tuning
# fine_tune_at = layer number where we start unfreezing layers

# configurations that will be used in training
# supported models: mobilenet_v2, efficientnet_v2, inception_v3, resnet_50
# mobilenet (contains 154 layers)
# efficientnet_v2 (contains 513 layers)
# inception_v3 (contains 311 layers)
# resnet_50 (contains 175 layers)
configs = [
    {"model_name": "mobilenet", "learning_rate": 0.001, "batch_size": 32, "image_size" : 224, "optimizer": "adam", "epochs": 50, "save_metrics": True, "folds": 5, "fine_tune": False, "fine_tune_epochs": 25, "fine_tune_at": 150},
    # {"model_name": "mobilenet", "learning_rate": 0.001, "batch_size": 32, "image_size" : 224, "optimizer": "adam", "epochs": 50, "save_metrics": True, "folds": 5, "fine_tune": True, "fine_tune_epochs": 25, "fine_tune_at": 148},
    # {"model_name": "mobilenet", "learning_rate": 0.001, "batch_size": 32, "image_size" : 224, "optimizer": "adam", "epochs": 50, "save_metrics": True, "folds": 5, "fine_tune": False, "fine_tune_epochs": 25, "fine_tune_at": 150},
    # {"model_name": "mobilenet", "learning_rate": 0.001, "batch_size": 32, "image_size" : 224, "optimizer": "adam", "epochs": 50, "save_metrics": True, "folds": 5, "fine_tune": True, "fine_tune_epochs": 25, "fine_tune_at": 148},

    # {"model_name": "efficientnet", "learning_rate": 0.001, "batch_size": 32, "image_size" : 224, "optimizer": "adam", "epochs": 50, "save_metrics": True, "folds": 5, "fine_tune": False, "fine_tune_epochs": 25, "fine_tune_at": 150},
    
    # {"model_name": "densenet", "learning_rate": 0.001, "batch_size": 32, "image_size" : 224, "optimizer": "adam", "epochs": 50, "save_metrics": True, "folds": 5, "fine_tune": False, "fine_tune_epochs": 25, "fine_tune_at": 150},

    # {"model_name": "inceptionv3", "learning_rate": 0.001, "batch_size": 32, "image_size" : 224, "optimizer": "adam", "epochs": 50, "save_metrics": True, "folds": 5, "fine_tune": False, "fine_tune_epochs": 25, "fine_tune_at": 150},

    # {"model_name": "resnet50", "learning_rate": 0.001, "batch_size": 32, "image_size" : 224, "optimizer": "adam", "epochs": 50, "save_metrics": True, "folds": 5, "fine_tune": False, "fine_tune_epochs": 25, "fine_tune_at": 150},

    # {"model_name": "vgg16", "learning_rate": 0.001, "batch_size": 32, "image_size" : 224, "optimizer": "adam", "epochs": 50, "save_metrics": True, "folds": 5, "fine_tune": False, "fine_tune_epochs": 25, "fine_tune_at": 15

    # {"model_name": "xception", "learning_rate": 0.001, "batch_size": 32, "image_size" : 224, "optimizer": "adam", "epochs": 50, "save_metrics": True, "folds": 5, "fine_tune": False, "fine_tune_epochs": 25, "fine_tune_at": 150},
]

# Define the base path for saving models
save_dir = "../saved_models"
os.makedirs(save_dir, exist_ok=True)

## Training 

### Setup

In [48]:
# Load dataset without splitting
dataset = tf.keras.utils.image_dataset_from_directory(
    data_root,                                  # loads images from the data_root directory
    image_size=(img_height, img_width),         # resizes all images to (224, 224) pixels
    batch_size=batch_size,                      # set the batch size
    shuffle=True                                # shufle data when loaded
)

class_names = np.array(dataset.class_names)     # get the class names for the data
num_classes = len(class_names)                  # get the number of classes in the dataset

# convert the dataset to a list of (image, label) pairs. This makes it easier to perform cross-validation
image_paths, labels = [], []
for image_batch, label_batch in dataset:
    image_paths.extend(image_batch.numpy())
    labels.extend(label_batch.numpy())

image_paths = np.array(image_paths)             # convert to numpy array to facilitate training
labels = np.array(labels)                       # convert to numpy array to facilitate training

# Split the dataset into training/validation and test sets
train_val_images, test_images, train_val_labels, test_labels = train_test_split(
    image_paths, labels, test_size=0.10, random_state=42, stratify=labels
)

def callbacks_setup(checkpoint_filepath):
    # EarlyStopping callback configuration
    early_stopping = EarlyStopping(
        monitor='val_loss',        # monitor validation loss
        patience=10,               # number of epochs with no improvement to stop training
        mode='min',                # want to minimize what it being monitored 
        restore_best_weights=False # don't restore in EarlyStopping, handled by ModelCheckpoint
    )

    model_checkpoint = ModelCheckpoint(
        filepath=checkpoint_filepath,   # path to save weights
        save_weights_only=True,         # only save weights instead of full model
        monitor='val_recall',           # monitor validation loss
        mode='max',                     # want to maximize what is being monitored
        save_best_only=True             # save the best weights
    )            

    reduce_lr = ReduceLROnPlateau(
        monitor='val_loss',      # monitor validation loss 
        factor=0.5,              # factor by which the learning rate will be reduced 
        patience=4,              # number of epochs with no improvement to stop training 
        mode='min',              # want to minimize what it being monitored 
        min_lr=1e-6              # lower bound on the learning rate 
    )            

    return early_stopping, model_checkpoint, reduce_lr

NotFoundError: Could not find directory ../data/Augmented_Images

### Metrics

In [49]:
from sklearn.metrics import precision_score, classification_report, roc_auc_score, recall_score, f1_score, confusion_matrix, ConfusionMatrixDisplay

# plot and save confusion matrix
def save_confusion_matrix(true_labels, predicted_labels, class_names, save_path):
    cm = confusion_matrix(true_labels, predicted_labels)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
    disp.plot(cmap=plt.cm.Blues)
    plt.title("Confusion Matrix")
    plt.savefig(save_path)
    plt.close()

# plot and save loss curves
def save_loss_curve(history, save_path):
    plt.figure(figsize=(10, 6))
    plt.plot(history['loss'], label='Training Loss', color='blue')
    plt.plot(history['val_loss'], label='Validation Loss', color='orange')
    plt.title("Training and Validation Loss Over Epochs")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.legend()
    plt.grid(True)
    plt.savefig(save_path)
    plt.close()

# compute and plot evaluation metrics (accuracy, sensitivity, specificity, F1 score)
def save_evaluation_metrics(true_labels, predicted_labels, cm, save_path): 

    accuracy = np.mean(predicted_labels == true_labels)  
    recall = recall_score(true_labels, predicted_labels, average='binary')     
    precision = precision_score(true_labels, predicted_labels, average='binary')   
    f1 = f1_score(true_labels, predicted_labels, average='binary')                
    roc_auc = roc_auc_score(true_labels, predicted_labels)

    # specificity = np.mean(np.diag(cm) / (np.diag(cm) + np.sum(cm, axis=0) - np.diag(cm)))   

    metrics = {
        "Accuracy": accuracy,
        "Precision": precision,
        "Sensitivity (Recall)": recall,
        # "Specificity: specificity,
        "F1-Score": f1,
        "ROC AUC": roc_auc
    }

    plt.figure(figsize=(10, 6))
    # Plot the metrics as bars
    bars = plt.bar(metrics.keys(), metrics.values(), color=['darkturquoise', 'sandybrown', 'hotpink', 'limegreen', 'mediumpurple'])

    # Annotate each bar with the corresponding value
    for bar in bars:
        yval = bar.get_height()
        plt.text(bar.get_x() + bar.get_width() / 2, yval, f'{yval:.4f}', ha='center', va='bottom')  # Add metric value at the top of the bar

    plt.title("Model Evaluation Metrics")
    plt.ylim([0, 1])
    plt.yticks(np.arange(0, 1.1, 0.1))
    plt.ylabel("Score")
    plt.savefig(save_path)
    plt.close()
    return metrics

# save classification report
def save_classification_report(true_labels, predicted_labels, class_names, save_path):
    class_report = classification_report(true_labels, predicted_labels, target_names=class_names, digits=4)
    with open(save_path, "w") as f:
        f.write(class_report)

# Function to calculate metrics
def calculate_metrics(true_labels, predictions):
    # Apply threshold of 0.5 to convert probabilities to binary predictions
    predicted_labels = (predictions > 0.5).astype(int).flatten()

    accuracy = np.mean(predicted_labels == true_labels)
    precision = precision_score(true_labels, predicted_labels, average='binary')  
    recall = recall_score(true_labels, predicted_labels, average='binary')        
    f1 = f1_score(true_labels, predicted_labels, average='binary')                
    auc = roc_auc_score(true_labels, predictions)                                 

    return accuracy, precision, recall, f1, auc

# Function to save metrics, loss curve, and confusion matrix for the best model
def save_best_model_visuals(history, model, val_ds, class_names, weights_path, fold):
    # Generate predictions for the validation set
    val_predictions = model.predict(val_ds)
    val_predicted_probs = val_predictions.flatten()  # Convert 2D predictions into 1D probabilities
    val_predicted_ids = (val_predicted_probs > 0.5).astype(int)  # Convert probabilities to binary predictions
    true_labels = np.concatenate([y for x, y in val_ds], axis=0)

    # Confusion Matrix
    confusion_matrix_path = os.path.join(weights_path, "confusion_matrix_fold.png")
    save_confusion_matrix(true_labels, val_predicted_ids, class_names, confusion_matrix_path)

    # Loss Curve
    loss_curve_path = os.path.join(weights_path, "loss_curve_fold.png")
    save_loss_curve(history.history, loss_curve_path)

    # Evaluation Metrics (Accuracy, Sensitivity, Specificity, F1 Score)
    cm = confusion_matrix(true_labels, val_predicted_ids)
    metrics_bar_chart_path = os.path.join(weights_path, "evaluation_metrics_fold.png")
    save_evaluation_metrics(true_labels, val_predicted_ids, cm, metrics_bar_chart_path)

    # Save classification report as a text file
    classification_report_path = os.path.join(weights_path, "classification_report_fold.txt")
    save_classification_report(true_labels, val_predicted_ids, class_names, classification_report_path)


### Automated Hyperparameter Tuning

In [ ]:
# # Split data into training/validation set for hyperparameter tuning
# train_images_tuning, val_images_tuning, train_labels_tuning, val_labels_tuning = train_test_split(
#     image_paths, labels, test_size=0.1, random_state=42, stratify=labels
# )

# # Define the hypermodel for hyperparameter tuning
# def build_model(hp):
#     base_model = tf.keras.applications.MobileNetV2(
#         input_shape=(img_height, img_width, 3),
#         include_top=False,
#         weights='imagenet'
#     )
#     base_model.trainable = False  # Freeze layers initially
    
#     model = Sequential([
#         base_model,
#         layers.GlobalAveragePooling2D(),
#         layers.Dense(num_classes)
#     ])

#     # Tune hyperparameters
#     learning_rate = hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='log')
#     optimizer = hp.Choice('optimizer', values=['adam', 'sgd'])

#     if optimizer == 'adam':
#         opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)
#     else:
#         opt = tf.keras.optimizers.SGD(learning_rate=learning_rate)

#     model.compile(
#         optimizer=opt,
#         loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
#         metrics=['accuracy']
#     )
    
#     return model

# # Set up the tuner for hyperparameter tuning
# tuner = kt.RandomSearch(
#     build_model,
#     objective='val_accuracy',  # Optimize for validation accuracy
#     max_trials=10,             # Try 10 different hyperparameter combinations
#     executions_per_trial=1,    # Run each combination once
#     directory='hyperparameter_tuning',
#     project_name='best_hyperparams_tuning'
# )

# # Prepare TensorFlow datasets for training and validation
# train_ds = tf.data.Dataset.from_tensor_slices((train_images_tuning, train_labels_tuning)).batch(batch_size)
# val_ds = tf.data.Dataset.from_tensor_slices((val_images_tuning, val_labels_tuning)).batch(batch_size)

# # Perform the hyperparameter search on the validation set
# tuner.search(train_ds, validation_data=val_ds, epochs=10)

# # Get the best hyperparameters after the search
# best_hyperparams = tuner.get_best_hyperparameters(num_trials=1)[0]

# # Print the best hyperparameters
# print(f"Best Hyperparameters: {best_hyperparams.values}")


### Model creation and fine tuning

In [ ]:
# Function to create and compile the model
def create_model(num_classes, config, fine_tune=None):
    # if you are not fine tuning the model, instantiate a new model 
    if(fine_tune == False):         

        model_name = config["model_name"]

        if model_name == "mobilenet":
            base_model = tf.keras.applications.MobileNetV2
        elif model_name == "efficientnet":
            base_model = tf.keras.applications.EfficientNetB3
        elif model_name == "densenet":
            base_model = tf.keras.applications.DenseNet121
        elif model_name == "inceptionv3":
            base_model = tf.keras.applications.InceptionV3
        elif model_name == "resnet50":
            base_model = tf.keras.applications.ResNet50
        elif model_name == "vgg16":
            base_model = tf.keras.applications.VGG16
        elif model_name == "xception":
            base_model = tf.keras.applications.Xception
        else:
            raise ValueError(f"Model name '{model_name}' is not supported.")


        # instantiate mobilenet (contains 154 layers)
        base_model = base_model(
            input_shape=(img_height, img_width, 3),     # set the input it will receive
            include_top=False,                          # do not include top layer to perform transfer learning
            weights='imagenet'                          # load weights from imagenet dataset
        )
        base_model.trainable = False                    # Freeze the base model
        # print(len(base_model.layers))
        # add a layer in order to perform classification on our dataset
        model = Sequential([
            base_model,                         # use base_model as the start of your model
            layers.GlobalAveragePooling2D(),    # add a final layer to perform classification
            layers.Dense(1)                     # set the number of possible prediction to the num of classes in dataset for multiclass classification and 1 for binary classification
        ])
        
    # select optimizer and learning rate based on configuration
    if config["optimizer"] == "adam":
        optimizer = tf.keras.optimizers.Adam(learning_rate=config["learning_rate"])
    elif config["optimizer"] == "sgd":
        optimizer = tf.keras.optimizers.SGD(learning_rate=config["learning_rate"])
    else:
        raise ValueError(f"Unsupported optimizer: {config['optimizer']}")

    # compile the model
    model.compile(
        optimizer=optimizer,
        loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
        metrics=['accuracy', tf.keras.metrics.Precision(name='precision'), tf.keras.metrics.Recall(name='recall')]
    )
    
    return model

# fine tune model by unfreezing the layers after the first fine_tune_at layers
def fine_tune_model(base_model, fine_tune_at):
    # Unfreeze the layers starting from fine_tune_at index
    for layer in base_model.layers[:fine_tune_at]:
        layer.trainable = False
    for layer in base_model.layers[fine_tune_at:]:
        layer.trainable = True


### Training loop

In [ ]:
train_metrics = []      # list to save training metrics
val_metrics = []        # list to save validation metrics
normalization_layer = layers.Rescaling(1.0 / 255)
test_ds = tf.data.Dataset.from_tensor_slices((test_images, test_labels))
test_ds = test_ds.map(lambda x, y: (normalization_layer(x), y)).batch(32)

for i, config in enumerate(configs):
    print(f"Training model {i + 1}/{len(configs)} with config: {config}")

    # K-fold Cross Validation
    kfold = StratifiedKFold(n_splits=config['folds'], shuffle=True, random_state=42)
    best_val_f1score = -float('inf')            # Initialize best F1 score with a very low value

    # Define the base path for saving models
    model_subdir = os.path.join(save_dir, f'model{i + 1}')
    os.makedirs(model_subdir, exist_ok=True)

    # Define the base path for saving checkpoints for model
    checkpoint_folder = os.path.join(model_subdir, 'checkpoints')
    os.makedirs(checkpoint_folder, exist_ok=True)

    # Define the base path for saving cthe model with the best f1-score
    best_f1_dir = os.path.join(model_subdir, 'best_f1score_fold')
    os.makedirs(best_f1_dir, exist_ok=True)
    
    # Training and validation loop for each fold
    fold = 1
    for train_idx, val_idx in kfold.split(train_val_images, train_val_labels):
        print(f"\nFold {fold}/{config['folds']}...")

        checkpoint_filepath = os.path.join(checkpoint_folder, f'checkpoint_fold{fold}.weights.h5')

        # Create subset datasets for training and validation
        train_images, train_labels = train_val_images[train_idx], train_val_labels[train_idx]
        val_images, val_labels = train_val_images[val_idx], train_val_labels[val_idx]

        # Convert NumPy arrays back to TensorFlow datasets
        train_ds = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
        val_ds = tf.data.Dataset.from_tensor_slices((val_images, val_labels))

        # Normalize datasets 
        normalization_layer = layers.Rescaling(1./255)
        train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
        val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))

        # prefetch data to improve performance by overlapping data preprocessing and model execution and cache the dataset in memory and batch
        AUTOTUNE = tf.data.AUTOTUNE
        train_ds = train_ds.batch(batch_size).cache().prefetch(buffer_size=AUTOTUNE)
        val_ds = val_ds.batch(batch_size).cache().prefetch(buffer_size=AUTOTUNE)

        # Step 1: Train model with frozen layers
        print(f"Training with frozen base layers for {config['epochs']} epochs...")

        # Create and compile model for each fold
        model = create_model(num_classes, config, fine_tune=False) 

        # setup callbacks 
        early_stopping, model_checkpoint, reduce_lr = callbacks_setup(checkpoint_filepath)

        # train the model on the training set until the epochs specified
        history_frozen = model.fit(
            train_ds,                                       # dataset used for training
            validation_data=val_ds,                         # dataset used for validation
            epochs=config['epochs'],                        # epochs used for training
            callbacks=[early_stopping, model_checkpoint, reduce_lr],   # set early stopping to avoid overfitting
            verbose=1
        ) 

        # load the best weights from ModelCheckpoint after training
        model.load_weights(checkpoint_filepath)

        if(config["fine_tune"] == True):
            # Step 2: Unfreeze layers and fine-tune
            print(f"Unfreezing layers starting from layer {config['fine_tune_at']} for fine-tuning...")
            fine_tune_model(model.layers[0], config['fine_tune_at'])      # fine tune model

            # re-compile the model with a lower learning rate for fine-tuning
            fine_tune_lr = config['learning_rate'] * 0.01

            model.compile(
                optimizer=tf.keras.optimizers.Adam(learning_rate=fine_tune_lr),
                loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                metrics=['accuracy', tf.keras.metrics.Precision(name='precision'), tf.keras.metrics.Recall(name='recall')]
            )
                
            print(f"Fine-tuning for {config['fine_tune_epochs']} epochs...")

            # setup callbacks again for fine-tuning phase with a unique checkpoint
            early_stopping, model_checkpoint, reduce_lr = callbacks_setup(checkpoint_filepath)
            
            history_fine_tune = model.fit(
                train_ds,                                       # dataset used for training
                validation_data=val_ds,                         # dataset used for validation
                epochs=config['fine_tune_epochs'],                        # epochs used for training
                callbacks=[early_stopping, model_checkpoint, reduce_lr],   # set early stopping to avoid overfitting
                verbose=1
            )

            # load weights after fine-tuning
            model.load_weights(checkpoint_filepath)

        # evaluate on validation set after training
        val_predictions = model.predict(val_ds)
        avg_val_loss = model.evaluate(val_ds, verbose=0)[0]
        avg_val_accuracy, avg_val_precision, avg_val_recall, avg_val_f1, avg_val_auc = calculate_metrics(
            np.concatenate([y for x, y in val_ds]), val_predictions
        )

        print(f"\nValidation: \tFold {fold} - Loss: {avg_val_loss:.4f}, Accuracy: {avg_val_accuracy:.4f}, Precision: {avg_val_precision:.4f}, Recall: {avg_val_recall:.4f}, F1 Score: {avg_val_f1:.4f}, ROC AUC Score: {avg_val_auc:.4f}")

        test_predictions = model.predict(test_ds)
        predicted_labels = np.argmax(test_predictions, axis=-1)
        true_labels = np.concatenate([y for _, y in test_ds], axis=0)

        avg_test_f1 = f1_score(true_labels, predicted_labels, average='macro')

        # save the best model based on validation F1 score
        if avg_test_f1 > best_val_f1score:
            best_val_f1score = avg_test_f1

            # Save model in TensorFlow SavedModel format (for TensorRT compatibility)
            # model.save(best_f1_dir)
            # tf.saved_model.save(model, best_f1_dir)
            model.export(best_f1_dir)
            print(f"Model with best F1 score during Validation saved at Fold {fold} with F1 Score of {best_val_f1score:.4f}")

            if (config['save_metrics'] == True):
                #save confusion matrix, loss curve, evaluation metrics for the best model
                history = history_frozen
                save_best_model_visuals(history, model, test_ds, class_names, model_subdir, fold)

        fold += 1       # Move to the next fold

Training model 1/1 with config: {'learning_rate': 0.001, 'optimizer': 'adam', 'epochs': 50, 'save_metrics': True, 'folds': 5, 'fine_tune': False, 'fine_tune_epochs': 25, 'fine_tune_at': 150}

Fold 1/5...
Training with frozen base layers for 50 epochs...
Epoch 1/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 29s 304ms/step - accuracy: 0.6290 - loss: 0.6206 - precision: 0.7218 - val_accuracy: 0.7983 - val_loss: 0.4485 - val_precision: 0.8507 - learning_rate: 0.0010
Epoch 2/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 17s 241ms/step - accuracy: 0.8073 - loss: 0.4042 - precision: 0.8754 - val_accuracy: 0.8330 - val_loss: 0.3846 - val_precision: 0.8725 - learning_rate: 0.0010
Epoch 3/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 18s 247ms/step - accuracy: 0.8533 - loss: 0.3427 - precision: 0.9049 - val_accuracy: 0.8626 - val_loss: 0.3487 - val_precision: 0.8867 - learning_rate: 0.0010
Epoch 4/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 18s 247ms/step - accuracy: 0.8824 - loss: 0.3054 - precision: 0.9161 - val_accuracy: 0.8730 - val_loss: 0.3241 - val_

## Testing

In [ ]:
for i, config in enumerate(configs):
    # model = tf.keras.models.load_model(f'../saved_models/model{i+1}/best_f1score_fold')

    # Load the SavedModel using TFSMLayer, treating it as a Keras layer
    model_layer = tf.keras.layers.TFSMLayer(f'../saved_models/model{i+1}/best_f1score_fold', call_endpoint='serving_default')
    
    # Wrap the TFSMLayer in a Sequential model for inference
    model = tf.keras.Sequential([model_layer])

    # once training is complete, evaluate on the held-out test set
    print(f"Evaluating the best model for model{i+1} on the held-out test set...")
    test_ds = tf.data.Dataset.from_tensor_slices((test_images, test_labels))
    test_ds = test_ds.map(lambda x, y: (normalization_layer(x), y)).batch(batch_size)

    test_predictions = model.predict(test_ds)

    # Print the shape and type of predictions for debugging
    # print(f"Predictions shape: {len(test_predictions['output_0'])}, type: {type(test_predictions)}")
    # print(f"First few predictions: {test_predictions[:5]}")  # Check the first few predictions

    # avg_test_loss = model.evaluate(test_ds, verbose=0)[0]
    avg_test_accuracy, avg_test_precision, avg_test_recall, avg_test_f1, avg_test_auc = calculate_metrics(
        np.concatenate([y for x, y in test_ds]), test_predictions['output_0']
    )

    print(f"\nTest Set Evaluation - Accuracy: {avg_test_accuracy:.4f}, Precision: {avg_test_precision:.4f}, Recall: {avg_test_recall:.4f}, F1 Score: {avg_test_f1:.4f}, AUC Score: {avg_test_auc:.4f}\n")


Evaluating the best model for model1 on the held-out test set...
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 58ms/step

Test Set Evaluation - Accuracy: 0.8906, Precision: 0.8944, Recall: 0.9096, F1 Score: 0.9020, AUC Score: 0.9681

Evaluating the best model for model2 on the held-out test set...
20/20 ━━━━━━━━━━━━━━━━━━━━ 10s 464ms/step

Test Set Evaluation - Accuracy: 0.7344, Precision: 0.7190, Recall: 0.8531, F1 Score: 0.7804, AUC Score: 0.7584



KeyboardInterrupt: 